In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [2]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [3]:
x = ['CholayilManagement','DanoneMTManagement','GCPLFoodsManagement','GCPLManagement',
     'JohnsonJohnsonManagement','ParleManagement','ColgateManagement','PiramalManagement','ZydusMTManagement']

In [4]:
conn = pyodbc.connect(conn_str)

In [5]:
# StoreMaster_df = pd.DataFrame()

In [6]:
StoreMaster_df = pd.DataFrame()  # ensure initialized

for value in x:
    query2 = f"""
    SELECT DISTINCT
        '{value}' AS client,
        LISStoreCode,
        PERCENTILE_CONT(0.5) 
            WITHIN GROUP (ORDER BY CAST(InLatitude AS FLOAT)) 
            OVER (PARTITION BY LISStoreCode) AS MedianLatitude,
        PERCENTILE_CONT(0.5) 
            WITHIN GROUP (ORDER BY CAST(InLongitude AS FLOAT)) 
            OVER (PARTITION BY LISStoreCode) AS MedianLongitude
    FROM [PowerBI_LISV2].[{value}].[ISPStoreAttendance_NormalExport]
    WHERE LISStoreCode NOT LIKE '%SP'
      AND LOWER(LISStoreCode) NOT LIKE '%test%' and InLatitude is not null;
    """

    df = pd.read_sql(query2, conn)
    StoreMaster_df = pd.concat([StoreMaster_df, df], ignore_index=True)

    print(f"Running for: {value}")


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8200\1153471199.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


Running for: CholayilManagement
Running for: DanoneMTManagement
Running for: GCPLFoodsManagement
Running for: GCPLManagement
Running for: JohnsonJohnsonManagement
Running for: ParleManagement
Running for: ColgateManagement
Running for: PiramalManagement
Running for: ZydusMTManagement


In [7]:
StoreMaster_df.head()

client  LISStoreCode  MedianLatitude  MedianLongitude
0  CholayilManagement  Cholayil-001       28.661012        77.297247
1  CholayilManagement  Cholayil-002       31.292202        75.657991
2  CholayilManagement  Cholayil-003       25.135468        75.855925
3  CholayilManagement  Cholayil-004       27.220158        77.926982
4  CholayilManagement  Cholayil-005       28.970674        77.656521

In [8]:
StoreMaster_df.to_csv('StoreMaster_df.csv')

In [9]:
# chek for unique LIS Store code
StoreMaster_df['cnt'] = StoreMaster_df.groupby('LISStoreCode')['LISStoreCode'].transform('count')
StoreMaster_df_dup = StoreMaster_df[StoreMaster_df['cnt'] > 1]

In [10]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [11]:
# StoreMaster_df['MedianLatitude'] = StoreMaster_df['MedianLatitude'].astype(float)
# StoreMaster_df['MedianLongitude'] = StoreMaster_df['MedianLongitude'].astype(float)

In [12]:
# Convert lat-long to NumPy array
coords = StoreMaster_df[['MedianLatitude', 'MedianLongitude']].to_numpy()

In [13]:
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

In [14]:
db = DBSCAN(eps=0.000197, min_samples=5, metric='haversine').fit(coords_scaled)

In [15]:
db.labels_

array([   0,    1,   -1, ..., 1795,   -1,   -1], dtype=int64)

In [16]:
StoreMaster_df['cluster_id'] = db.labels_

In [17]:
StoreMaster_df.head()

client  LISStoreCode  MedianLatitude  MedianLongitude  cnt  \
0  CholayilManagement  Cholayil-001       28.661012        77.297247    1   
1  CholayilManagement  Cholayil-002       31.292202        75.657991    1   
2  CholayilManagement  Cholayil-003       25.135468        75.855925    1   
3  CholayilManagement  Cholayil-004       27.220158        77.926982    1   
4  CholayilManagement  Cholayil-005       28.970674        77.656521    1   

   cluster_id  
0           0  
1           1  
2          -1  
3           2  
4           3

In [18]:
StoreMaster_df.to_csv('StoreMaster_df2.csv')

In [19]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in KM
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [20]:
StoreMasterTable = pd.DataFrame()  # ensure initialized

for value in x:
    query2 = f"""
    SELECT
--     '{value}' AS client,
    ClientStoreCode,
    LISStoreCode,
    StoreName,
    ChainCode,
    StoreStatus,
    StoreState,
    StoreCity,
    StoreLatitude,
    StoreLongitude
FROM [PowerBI_LISV2].[{value}].StoreMaster
WHERE
    LOWER(LISStoreCode) NOT LIKE '%test%'
    AND LOWER(StoreName) NOT LIKE '%test%';
;
    """

    df = pd.read_sql(query2, conn)
    StoreMasterTable = pd.concat([StoreMasterTable, df], ignore_index=True)

    print(f"Running for: {value}")


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8200\3713691969.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8200\3713691969.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


Running for: CholayilManagement
Running for: DanoneMTManagement
Running for: GCPLFoodsManagement
Running for: GCPLManagement
Running for: JohnsonJohnsonManagement
Running for: ParleManagement
Running for: ColgateManagement
Running for: PiramalManagement
Running for: ZydusMTManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8200\3713691969.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)


In [21]:
StoreMasterTable.head()

ClientStoreCode  LISStoreCode                          StoreName  \
0           10140  Cholayil-025  Shubham Store Pradhan tower Delhi   
1           10016  Cholayil-044           Reliance SB AGAR BAZAR\t   
2           10013  Cholayil-047     Reliance Ramchandra Lane Malad   
3           10044  Cholayil-051        Reliance Drive In Ahmedabad   
4            None  Cholayil-055                       Haji V Ahmad   

      ChainCode StoreStatus   StoreState  StoreCity StoreLatitude  \
0  StandaloneGT        None        Delhi      Delhi     28.606105   
1      Relaince        None  Maharashtra     Mumbai     19.018792   
2      Relaince        None  Maharashtra     Mumbai     19.188214   
3      Relaince        None      Gujarat  Ahmedabad     23.045922   
4  StandaloneGT      Active       Kerala  Kozhikode          None   

  StoreLongitude  
0      76.986541  
1      72.832995  
2      72.838439  
3      72.534103  
4           None

In [22]:
clustered_storemaster=pd.merge(StoreMaster_df,StoreMasterTable,how='left', on = 'LISStoreCode')

In [23]:
# clustered_storemaster.head()

In [24]:
cols = [
    "MedianLatitude", "MedianLongitude",
    "StoreLatitude", "StoreLongitude"
]

clustered_storemaster[cols] = clustered_storemaster[cols].apply(pd.to_numeric, errors="coerce")


clustered_storemaster["Distance_km"] = haversine(
    clustered_storemaster["MedianLatitude"],
    clustered_storemaster["MedianLongitude"],
    clustered_storemaster["StoreLatitude"],
    clustered_storemaster["StoreLongitude"]
)

In [25]:
clustered_storemaster.head()

client  LISStoreCode  MedianLatitude  MedianLongitude  cnt  \
0  CholayilManagement  Cholayil-001       28.661012        77.297247    1   
1  CholayilManagement  Cholayil-002       31.292202        75.657991    1   
2  CholayilManagement  Cholayil-003       25.135468        75.855925    1   
3  CholayilManagement  Cholayil-004       27.220158        77.926982    1   
4  CholayilManagement  Cholayil-005       28.970674        77.656521    1   

   cluster_id ClientStoreCode                  StoreName ChainCode  \
0           0           10086   Metro CNC Shahdara Delhi     Metro   
1           1           10096      Walmart CNC Jalandhar   Walmart   
2          -1           10076           Walmart CNC Kota   Walmart   
3           2           10095  Walmart CNC Sikander Agra   Walmart   
4           3           10229           Metro CNC Meerut     Metro   

  StoreStatus     StoreState  StoreCity  StoreLatitude  StoreLongitude  \
0        None          Delhi      Delhi      28.660965       77.297021   
1        None         Punjab  Jalandhar      31.292454       75.657830   
2        None      Rajasthan       Kota      25.134903       75.855757   
3        None  Uttar Pradesh       Agra      27.220065       77.927055   
4        None  Uttar Pradesh     Meerut      28.950086       77.454242   

   Distance_km  
0     0.022654  
1     0.031882  
2     0.065013  
3     0.012634  
4    19.812563

In [26]:
clustered_storemaster.to_csv('clustered_storemaster.csv')

In [30]:
revised_clustered_storemaster = pd.read_csv('revised_clustered_storemaster.csv')

In [31]:
revised_clustered_storemaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32904 entries, 0 to 32903
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            32904 non-null  int64  
 1   client                32904 non-null  object 
 2   LISStoreCode          32904 non-null  object 
 3   MedianLatitude        32904 non-null  float64
 4   MedianLongitude       32904 non-null  float64
 5   cnt                   32904 non-null  int64  
 6   cluster_id            32904 non-null  int64  
 7   ClientStoreCode       18013 non-null  object 
 8   StoreName             31530 non-null  object 
 9   ChainCode             31530 non-null  object 
 10  StoreStatus           11266 non-null  object 
 11  StoreState            31530 non-null  object 
 12  StoreCity             31530 non-null  object 
 13  StoreLatitude         18769 non-null  float64
 14  StoreLongitude        18769 non-null  float64
 15  Distance_km        

In [32]:
# Convert lat-long to NumPy array
coords = revised_clustered_storemaster[['final_StoreLatitude', 'final_StoreLongitude']].to_numpy()

In [33]:
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

In [34]:
db = DBSCAN(eps=0.000197, min_samples=5, metric='haversine').fit(coords_scaled)

In [35]:
revised_clustered_storemaster['cluster_id2'] = db.labels_

In [36]:
revised_clustered_storemaster.to_csv('revised_clustered_storemaster2.csv')